<a href="https://colab.research.google.com/github/ElenaVillano/sentiment_analysis_tweets/blob/main/notebooks/limpieza_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import nltk
import re
import timeit
import string
#testing commit 

#call the nltk downloader
nltk.download('punkt')

from dateutil import parser

from sklearn.model_selection import train_test_split

# Carga un set de stopwords predefinidas
from nltk.corpus import stopwords

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Nombramiento de columnas
col_names = ['target', # Polaridad del twet 0=negativo, 2=neutral, 4=positivo
             'ids',    # ID tweet
             'date',   # Fecha y hora del tweet
             'flag',   # QUERY
             'user',   # Usuario del tweet
             'text']   # Texto del tweet

# Carga y limpieza de datos

In [ ]:
# Requiered to select a file to be imported into colab
# Not useful if running locally
from google.colab import files
uploaded = files.upload()

Saving training.1600000.processed.noemoticon.csv to training.1600000.processed.noemoticon.csv
Saving testdata.manual.2009.06.14.csv to testdata.manual.2009.06.14.csv


In [ ]:
training = pd.read_csv('training.1600000.processed.noemoticon.csv',
                 encoding='latin-1', names=col_names)

In [ ]:
test = pd.read_csv('testdata.manual.2009.06.14.csv', names=col_names)

In [ ]:
print(training.shape)
print(test.shape)

(1600000, 6)
(498, 6)


In [ ]:
training.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# Revisamos si tenemos valores nulos.
print("Revisamos si hay valores nulos en el set de entrenamiento\n", training.isna().sum())

Revisamos si hay valores nulos en el set de entrenamiento
 target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64


In [ ]:
# Revisamos si tenemos valores nulos.
print("Revisamos si hay valores nulos en el set de pruebas\n", test.isna().sum())

Revisamos si hay valores nulos en el set de pruebas
 target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64


## Convertimos a minúsculas

In [ ]:
def convierte_a_minusculas(df):
    """
    Convierte a minúsculas todas las celdas de tipo str en un dataframe
    ==========
    * Args:
         - df: dataframe
    * Return:
         - df: dataframe en minúsculas
    ==========
    Ejemplo:
        >> dataframe = convierte_a_minusculas(dataframe)
    """

    df = df.applymap(lambda s: s.lower() if type(s) == str else s)

    return df

In [ ]:
training = convierte_a_minusculas(training)
test = convierte_a_minusculas(test)

## Reemplazamos las URLs con el texto "URL"

In [ ]:
#The regular expression used to match URLs is ((www\.[\S]+)|(https?://[\S]+)).

def reemplazar_URLs(phrase):
  return re.sub("((www\.[\S]+)|(https?://[\S]+))", "url", phrase)

In [ ]:
training['text'] = training['text'].map(lambda s: reemplazar_URLs(s))
test['text'] = test['text'].map(lambda s: reemplazar_URLs(s))

## Reemplazamos las menciones @ con la palabra "USER_MENTION"



In [ ]:
# The regular expression (regex) used to match user mention is @[\S]+. 2.

In [ ]:
def reemplazar_usuarios(phrase):
  return re.sub("@[\S]+", "user_mention", phrase)

In [ ]:
training['text'] = training['text'].map(lambda s: reemplazar_usuarios(s))
test['text'] = test['text'].map(lambda s: reemplazar_usuarios(s))

## Quitamos el Hashtag , pero dejamos la palabra

In [ ]:
# The regular expression used to match hashtags is #(\S+).

In [ ]:
def quitar_hashtag(phrase):
  return re.sub("#(\S+)", "", phrase)

In [ ]:
training['text'] = training['text'].map(lambda s: quitar_hashtag(s))
test['text'] = test['text'].map(lambda s: quitar_hashtag(s))

## Quitamos los Retweets, sólo la palabra RT , dejamos el comentario

In [ ]:
#  The regular expression used to match retweets is \brt\b.

In [ ]:
def quitar_RT(phrase):
  return re.sub("\brt\b", "", phrase)

In [ ]:
training['text'] = training['text'].map(lambda s: quitar_RT(s))
test['text'] = test['text'].map(lambda s: quitar_RT(s))

## Quitamos espacios o puntos extras

In [ ]:
## Reemplazamos los dobles puntos (o más) con un espacio , dos o más espacios con 1 espacio y hacemos strip de espacios y comillas
# Strip any punctuation [’"?!,.():;-'] from the word ??

In [ ]:
def quitar_caracteres_especiales(phrase):
  caracteres_especiales = '’"?!,.():;-'
  # quita dobles espacios y lo hace 1
  phrase = re.sub(' +', ' ', phrase)
  # quita los caracteres especiales
  regex = re.compile('[%s]' % re.escape(caracteres_especiales))

  return regex.sub('', phrase)

In [ ]:
training['text'] = training['text'].map(lambda s: quitar_caracteres_especiales(s))
test['text'] = test['text'].map(lambda s: quitar_caracteres_especiales(s))

## Quitamos expresiones con letras repetidas

In [ ]:
#Convert 2 or more letter repetitions to 2 letters.
#Some people send tweets like I am sooooo
#happpppy adding multiple characters to emphasize
#on certain words. This is done to handle such tweets
#by converting them to I am soo happy

In [ ]:
def quitar_letras_repetidas(phrase):
  return re.sub(r'(.)\1+', r'\1\1', phrase)

In [ ]:
training['text'] = training['text'].map(lambda s: quitar_letras_repetidas(s))
test['text'] = test['text'].map(lambda s: quitar_letras_repetidas(s))

## Quitar caracteres nonascii

In [ ]:
def quitar_nonascii(phrase):
  return phrase.encode('ascii', errors='ignore').decode('utf-8')

In [ ]:
training['text'] = training['text'].map(lambda s: quitar_nonascii(s))
test['text'] = test['text'].map(lambda s: quitar_nonascii(s))

## Removemos stopwords

In [ ]:
nltk.download('stopwords') # Use predefined list of stopwords from nltk

STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Aux funcción para substituir abreviaciones
def decontracted(phrase):
  phrase = re.sub(r"won't", "will not", phrase)
  phrase = re.sub(r"can\'t", "can not", phrase)
  phrase = re.sub(r"n\'t", " not", phrase)
  phrase = re.sub(r"\'re", " are", phrase)
  phrase = re.sub(r"\'s", " is", phrase)
  phrase = re.sub(r"\'d", " would", phrase)
  phrase = re.sub(r"\'ll", " will", phrase)
  phrase = re.sub(r"\'t", " not", phrase)
  phrase = re.sub(r"\'ve", " have", phrase)
  phrase = re.sub(r"\'m", " am", phrase)
  return phrase

In [ ]:
training['text'] = training['text'].map(lambda s: decontracted(s))
test['text'] = test['text'].map(lambda s: decontracted(s))

In [ ]:
# Set adicional de stopwords, removeremos palabras tanto de este set como del que
# viene definido en el nltk
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
# Función auxiliar para remover stopwords y otras estandarizaciones de texto
# (Aqui puedes incluir el lematizador o stemming si gustas explorarlos)
def remove_stopwords(phrase):
    phrase = ' '.join(e.lower() for e in phrase.split() if e.lower() not in stopwords)
    phrase = ' '.join(e.lower() for e in phrase.split() if e.lower() not in STOPWORDS)
    return phrase

In [ ]:
training['text'] = training['text'].map(lambda s: remove_stopwords(s))
test['atext'] = test['text'].map(lambda s: remove_stopwords(s))

In [ ]:
training

,target,ids,date,flag,user,text
0,0,1467810369,mon apr 06 22:19:45 pdt 2009,no_query,_thespecialone_,user_mention url aww bummer shoulda got david ...
1,0,1467810672,mon apr 06 22:19:49 pdt 2009,no_query,scotthamilton,upset update facebook texting might cry result...
2,0,1467810917,mon apr 06 22:19:53 pdt 2009,no_query,mattycus,user_mention dived many times ball managed sav...
3,0,1467811184,mon apr 06 22:19:57 pdt 2009,no_query,ellectf,whole body feels itchy like fire
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,user_mention behaving mad see
...,...,...,...,...,...,...
1599995,4,2193601966,tue jun 16 08:40:49 pdt 2009,no_query,amandamarie1028,woke school best feeling ever
1599996,4,2193601969,tue jun 16 08:40:49 pdt 2009,no_query,thewdboards,thewdbcom cool hear old walt interviews url
1599997,4,2193601991,tue jun 16 08:40:49 pdt 2009,no_query,bpbabe,ready mojo makeover ask details
1599998,4,2193602064,tue jun 16 08:40:49 pdt 2009,no_query,tinydiamondz,happy 38th birthday boo time tupac amaru shakur


## Stemming con NLTK

In [ ]:

#create an object of class PorterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [ ]:
training['text'] = training['text'].map(lambda s: stemSentence(s))
test['atext'] = test['text'].map(lambda s: stemSentence(s))

In [ ]:
training

,target,ids,date,flag,user,text
0,0,1467810369,mon apr 06 22:19:45 pdt 2009,no_query,_thespecialone_,user_ment url aww bummer shoulda got david car...
1,0,1467810672,mon apr 06 22:19:49 pdt 2009,no_query,scotthamilton,upset updat facebook text might cri result sch...
2,0,1467810917,mon apr 06 22:19:53 pdt 2009,no_query,mattycus,user_ment dive mani time ball manag save 50 % ...
3,0,1467811184,mon apr 06 22:19:57 pdt 2009,no_query,ellectf,whole bodi feel itchi like fire
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,user_ment behav mad see
...,...,...,...,...,...,...
1599995,4,2193601966,tue jun 16 08:40:49 pdt 2009,no_query,amandamarie1028,woke school best feel ever
1599996,4,2193601969,tue jun 16 08:40:49 pdt 2009,no_query,thewdboards,thewdbcom cool hear old walt interview url
1599997,4,2193601991,tue jun 16 08:40:49 pdt 2009,no_query,bpbabe,readi mojo makeov ask detail
1599998,4,2193602064,tue jun 16 08:40:49 pdt 2009,no_query,tinydiamondz,happi 38th birthday boo time tupac amaru shakur


Quitamos datos al entrenamiento, empecemos con 100,000 

In [ ]:
data_set_1 = training= training.head(100000)

In [ ]:
data_set_1

,target,ids,date,flag,user,text
0,0,1467810369,mon apr 06 22:19:45 pdt 2009,no_query,_thespecialone_,user_ment url aww bummer shoulda got david car...
1,0,1467810672,mon apr 06 22:19:49 pdt 2009,no_query,scotthamilton,upset updat facebook text might cri result sch...
2,0,1467810917,mon apr 06 22:19:53 pdt 2009,no_query,mattycus,user_ment dive mani time ball manag save 50 % ...
3,0,1467811184,mon apr 06 22:19:57 pdt 2009,no_query,ellectf,whole bodi feel itchi like fire
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,user_ment behav mad see
...,...,...,...,...,...,...
99995,0,1793820630,thu may 14 04:05:58 pdt 2009,no_query,babybigmouth,son develop new habit wake 530am second pot co...
99996,0,1793820812,thu may 14 04:06:00 pdt 2009,no_query,paramoreroxx,look like router broke tweet fone
99997,0,1793820858,thu may 14 04:06:00 pdt 2009,no_query,beckypope,realli dont want colleg right wish sunni
99998,0,1793820982,thu may 14 04:06:02 pdt 2009,no_query,xmissxxxmurderx,user_ment *offer pepto*


# Entrenamos un Tokenizer

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers

# Entrena un Tokenizer. Consiste en:
# Crea un diccionario numerado de las palabras existentes en el corpus, y devuelve
# por cada palabra, el número entero de su índice en el diccionario.
# En este caso, considera las N palabras más frecuentes.
# oov_token = constante asiganda para palabras fuera del vocabulario (NOT USED HERE)


max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data_set_1.text)
sequences = tokenizer.texts_to_sequences(data_set_1.text)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...  555 1577    6]
 [   0    0    0 ...   10  217  706]
 [   0    0    0 ...  357    3 2676]
 ...
 [   0    0    0 ...   60   20  369]
 [   0    0    0 ...    2    1 1207]
 [   0    0    0 ...   37   87  313]]


## Separamos en train, test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tweets, data_set_1.target.values, test_size=0.3, random_state=23042021)

In [ ]:
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (70000, 200)
y_train (70000,)
X_test (30000, 200)
y_test (30000,)


# Creamos el modelo

### RNN, tomado de: https://www.kaggle.com/smitshah00/sentiment-analysis-ml-rnn

In [ ]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint


model2 = Sequential()
model2.add(layers.Embedding(max_words, 128))
model2.add(layers.LSTM(64,dropout=0.5))
model2.add(layers.Dense(16, activation='relu'))
model2.add(layers.Dense(8, activation='relu'))
model2.add(layers.Dense(1,activation='sigmoid'))
model2.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

model2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 128)         640000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_12 (Dense)             (None, 16)                1040      
_________________________________________________________________
dense_13 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 9         
Total params: 690,593
Trainable params: 690,593
Non-trainable params: 0
_________________________________________________________________


In [ ]:
checkpoint2 = ModelCheckpoint("rnn_model.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test),callbacks=[checkpoint2])

Epoch 1/10
1176/2188 [===============>..............] - ETA: 2:23 - loss: 0.0328 - accuracy: 0.9998

# Probamos

In [ ]:
model = keras.models.load_model('rnn_model.hdf5')
sequence = tokenizer.texts_to_sequences(['this data science article is the best ever'])
test = pad_sequences(sequence, maxlen=max_len)
pred = model.predict(test)
if pred > 0.5:
  print('Positive')
else:
  print('Negative')